In [35]:
# Load data
import numpy as np

with open("../data/day20test.txt", "r") as f:
    input = f.read()

tiles = input.split("\n\n")

tile_dict = {}
for tile in tiles:
    lines = tile.split("\n")
    tile_id = int(lines[0].split(" ")[1][:-1])
    tile_data = np.array(
        [[int(character.replace(".", "0").replace("#", "1")) for character in line] for line in lines[1:]]
    )
    tile_dict[tile_id] = tile_data

#tile_dict

In [36]:
tile_dict[2311]

array([[0, 0, 1, 1, 0, 1, 0, 0, 1, 0],
       [1, 1, 0, 0, 1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 1, 1, 0, 0, 1, 0],
       [1, 1, 1, 1, 0, 1, 0, 0, 0, 1],
       [1, 1, 0, 1, 1, 0, 1, 1, 1, 0],
       [1, 1, 0, 0, 0, 1, 0, 1, 1, 1],
       [0, 1, 0, 1, 0, 1, 0, 0, 1, 1],
       [0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
       [1, 1, 1, 0, 0, 0, 1, 0, 1, 0],
       [0, 0, 1, 1, 1, 0, 0, 1, 1, 1]])

In [117]:
# total image can be flipped and rotated, so some degrees of freedom

# data structure:
# - ids, full arrays
#OR
# - ids, top, right, left, bottom line (rotation, flip)

# - grid with IDs

id_list = list(tile_dict.keys())

#grid[m,n] = doing
#get_available_edges(m,n)

# Grid Class
# Tile Class (wth method to return edges, etc.)

# prepare
# put one ID on grid
# update grid todo list (list of all neighbouring positions)
# update tile todo list (remove tile id)
# start
# select any position in grid todo list (iterate over them)
# get edges that must match, together with their position
#   - get neighbouring IDs
#   - get left side of right ID from dict, etc.
# find matching ID with rotation and flip by (itertools?)
# - iterate over IDs
#       - iterate over edges/rotation (4)
#           - iterate over flip (4)
# for every step, match 1-4 edges (right=right, top=top, etc.)
# if match:
# - update grid with id
# - update dict with rotation and flip (???)
#       - do it by actually flipping/rotating (or reordering lists)
# - update grid todo list
# - update tile todo list
# if no match:
# - put id to back of todo_list
# break if tile todo list is empty
# continue iterating over IDs

class Grid():
    """
    Main class for grid.
    """
    def __init__(self, tile_dict):
        self.tile_dict = tile_dict
        
        self.grid = np.zeros((5, 5))
        self.tile_todo = [] # list of ids
        self.grid_todo = [] # necessary, or calculate on the fly?

        self.tiles = [] # children?

        self.prepare()

    def prepare(self):

        # create tile todo
        self.tile_todo = list(self.tile_dict.keys())

        # update grid and tile todo
        start_coordinates = (2, 2)
        new_grid = self.grid
        new_grid[start_coordinates] = self.tile_todo.pop(0)
        new_grid[(3,3)] = self.tile_todo.pop(0)
        self.grid = new_grid

        # create first tile object

        # create grid todo
        self.grid_todo = self.get_adjacents()

    def get_adjacents(self):
        """
        Use gradient for edge detection.
        https://stackoverflow.com/questions/32334322/find-adjacent-elements-in-a-2d-numpy-grid
        """
        return self.grid


class Tile():
    """
    Main class for tile.

    needs id or not?
    """
    def __init__(self, tile_id, tile_data):
        self.tile_id = tile_id
        self.rotation = 0 #0,1,2,3
        self.flip = 0 # 0, 1, 2, 3 (to define what is what)
        self.tile_data = tile_data

    def set_orientation(self, rotation, flip):
        self.rotation = rotation
        self.flip = flip

    def get_edges(self):
        flip_y = self.flip in [1, 3]
        flip_x = self.flip in [2, 3]

        # put in logic for 
        list_of_edges = [
            self.tile_data[0,:], # top
            self.tile_data[-1,:], # bottom
            self.tile_data[:,0], # left
            self.tile_data[:,-1] # right
        ]

        list_of_edges_rotated = list_of_edges[self.rotation:] + list_of_edges[:self.rotation]
        # fix rotation
        
        return list_of_edges_rotated


grid = Grid(tile_dict)
grid1 = grid.get_adjacents()
grid1[(3,2)] = 1234
grid1[(3,1)] = 1234
grid1[(4,2)] = 1234
grid1[2:,2:] = 1234
print(grid1)
bla = np.nonzero(grid1) # returns x and y values separately
print(bla)
for positions in bla:
    print(positions)



[[   0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.]
 [   0.    0. 1234. 1234. 1234.]
 [   0. 1234. 1234. 1234. 1234.]
 [   0.    0. 1234. 1234. 1234.]]
(array([2, 2, 2, 3, 3, 3, 3, 4, 4, 4]), array([2, 3, 4, 1, 2, 3, 4, 2, 3, 4]))
[2 2 2 3 3 3 3 4 4 4]
[2 3 4 1 2 3 4 2 3 4]


In [106]:

dy, dx = np.gradient(grid1)
edges_x = np.abs(dx) > 0
edges_y = np.abs(dy) > 0
neighbour_array = (edges_x | edges_y) & ~(grid1 > 0)
print(neighbour_array)
coords = np.argwhere(neighbour_array)
print(coords)
print(coords[0])

# make testcase for get_adjacents

[[False False False False False]
 [False False  True  True  True]
 [False  True False False False]
 [ True False False False False]
 [False  True False False False]]
[[1 2]
 [1 3]
 [1 4]
 [2 1]
 [3 0]
 [4 1]]
[1 2]


In [ ]:
def get_available_edges(grid, coords):
    """
    Return list of positions that are empty.
    Order: top, bottom, left, right

    """
    neighbors = [
        grid[coords[0] - 1, coords[1]] == 0,
        grid[coords[0] + 1, coords[1]] == 0,
        grid[coords[0], coords[1] - 1] == 0,
        grid[coords[0], coords[1] + 1] == 0,
    ]
    return neighbors


In [58]:
grid = np.zeros((5, 5))
grid[1,2] = 1
grid[4, 2] = 4
print(grid)
m, n = 2, 2

get_available_edges(grid, (m, n))

[[0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 4. 0. 0.]]


[False, True, True, True]

In [116]:
x = [1,2,3,4,5]
n = 1
x[n:]+x[:n]

[2, 3, 4, 5, 1]

In [118]:
x[::-1]

[5, 4, 3, 2, 1]